In [21]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain import FewShotPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import DeepLake
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from chromadb.utils import embedding_functions
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import chromadb
from chromadb.config import Settings
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.chat_models import ChatOpenAI
from langchain import HuggingFaceHub

from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain.schema import SystemMessage, HumanMessage, AIMessage

from langchain.llms import GPT4All
# from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from tqdm import tqdm
from langchain.output_parsers import PydanticOutputParser
from pydantic import validator
from pydantic import BaseModel, Field
from typing import List
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

import openai
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
import cohere
from langchain.embeddings import CohereEmbeddings


In [22]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())


True
1
0


1. How to add more documents to chroma db?
2. What steps are taken when an agent is run?
3. What are the parameters that is responsible for the output any LLM generates? e.g: temperature, probability, etc.
4. Zero-shot vs few-shot prompts
5. Ways to ask multiple questions in one go?
6. How to use other open-source models apart from openai models?
7. Difference between .run(), .generate(), and .predict()?
8. Go through variuos chains in the presentation and how we can make these better with right prompts?
9. What is the difference between LLM and Chat models?
10. How to ask new questions to Chat model?
11. sampling: temp = 0.800000, top_k = 40, top_p = 0.950000, repeat_last_n = 64, repeat_penalty = 1.300000 generate: n_ctx = 512, n_batch = 1, n_predict = 256, n_keep = 0: \
    How to change arguments from above list for GPT4ALL model?
12. Various example selectors for Few shot prompts.
13. Making best use of GPT4ALL models.
14. Why large open-source models timw_out with HuggingFaceHub but working fine with HuggingFacePipeline?
15. What are different chain_type such as stuff,map_reduce, refine, etc.?
16. What is the use of LLMChainExtractor and ContextualCompressionRetriever?
17. Read about various data loaders:
    * including TextLoader, PyPDFLoader, SeleniumURLLoader, and Google Drive Loader.
18. Read about various Text splitters:
    * CharacterTextSplitter, RecursiveCharacterTextSplitter, NLTKTextSplitter, SpacyTextSplitter, MarkdownTextSplitter, TokenTextSplitter, etc.
19. Read about various embedding models/platforms:
    * OpenAI, HuggingFace, Cohere, etc.
20. Building a customer QA chatbot using walmart data
21. Avoid adding duplicate docs in chromadb index.
22. Read about various chains:
    * LLMChain, ConversationChain, SequentialChain, TransformationChain, LLMCheckerChain, LLMSummarizationChain, Custom Chain, etc.
23. Providing custom prompt to load_summarize_chain.



In [23]:
# import os
# from dotenv import load_dotenv, find_dotenv
# load_dotenv(find_dotenv())
# os.getenv('OPENAI_API_KEY')
# os.getenv('ACTIVELOOP_TOKEN')
# os.getenv('GOOGLE_API_KEY')
# os.getenv('GOOGLE_CSE_ID')

In [24]:
import os
os.environ['OPENAI_API_KEY'] = ""
os.environ["ACTIVELOOP_TOKEN"] = ""
os.environ["GOOGLE_API_KEY"]= ""
os.environ["GOOGLE_CSE_ID"]= ""
os.environ["HUGGINGFACEHUB_API_TOKEN"]= ""
os.environ["COHERE_API_KEY"] = ""
os.environ["WOLFRAM_ALPHA_APPID"] = ""
os.environ["SERPAPI_API_KEY"]= ""

In [25]:
# import os
os.environ['http_proxy'] = "http://sysproxy.wal-mart.com:8080"
os.environ['https_proxy'] = "http://sysproxy.wal-mart.com:8080"
# os.environ['SSL_CERT_FILE'] = '/etc/ssl/certs/ca-bundle.crt'
# os.environ['NO_PROXY'] = "*.walmart.com,*.wal-mart.com,*.walmart.net,*.wal-mart.net,localhost,127.0.0.1"

In [6]:
# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)

/libraries/llm_exp_1/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [ ]:
text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."
print(llm(text))

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.9)
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("eco-friendly water bottles"))

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

# Start the conversation
conversation.predict(input="Tell me about yourself.")

# Continue the conversation
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with data analysis?")

# Display the conversation
print(conversation)

In [ ]:
# instantiate the LLM and embeddings models
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
# embeddings = embedding_functions.OpenAIEmbeddingFunction(model_name="text-embedding-ada-002", api_key=os.environ.get('OPENAI_API_KEY'))
# embeddings = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")




# create our documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# load it into Chroma
# db = Chroma.from_documents(docs, embeddings)


In [ ]:
# db.get(include=['embeddings', 'documents'])

In [ ]:
# import chromadb
# persistent_client = chromadb.Client()
# collection = persistent_client.get_or_create_collection("collection_name")
# collection.add(ids=["1", "2"], documents= docs)

# langchain_chroma = Chroma(
#     client=persistent_client,
#     collection_name="collection_name",
#     embedding_function=embeddings,
# )

# print("There are", langchain_chroma._collection.count(), "in the collection")

In [ ]:
tmp_client = chromadb.Client()
db_openai = Chroma.from_documents(
    docs, embeddings, client=tmp_client, collection_name="openai_collection"
)

print("There are", db_openai._collection.count(), "in the collection")



In [ ]:
# db_openai.get(include=['documents'])

In [ ]:
# tmp_client.get_collection('openai_collection')


In [ ]:
retrieval_qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db_openai.as_retriever())

In [ ]:
tools = [
    Tool(
    name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
response = agent.run("When was Napoleone born?")
print(response)

In [ ]:
# create new documents
texts = [
    "Lady Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)


In [ ]:
# openai_lc_client.get()

In [ ]:
# openai_lc_client.add_documents(docs, embeddings, client=new_client, collection_name="openai_collection")
db_openai= Chroma.from_documents(
    docs, embeddings, client=tmp_client, collection_name="openai_collection"
)
print("There are", db_openai._collection.count(), "in the collection")


In [ ]:
response = agent.run("When was Michael Jordan born?")
print(response)

In [ ]:
# db_openai.get(include=['documents'])

In [ ]:
response = agent.run("Who is Lady Gaga")
print(response)

In [ ]:
# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

In [ ]:
tools = [
    Tool(
        name = "google-search",
        func=search.run,
        description="useful for when you need to search google to answer questions about current events"
    )
]

agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True,
                         max_iterations=6)


response = agent("What's the latest news about the Mars rover?")
print(response['output'])

In [ ]:

prompt = PromptTemplate(
    input_variables=["query"],
    template="Write a summary of the following text: {query}"
)

summarize_chain = LLMChain(llm=llm, prompt=prompt)

# remember to set the environment variables
# “GOOGLE_API_KEY” and “GOOGLE_CSE_ID” to be able to use
# Google Search via API.
search = GoogleSearchAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for finding information about recent events"
    ),
    Tool(
       name='Summarizer',
       func=summarize_chain.run,
       description='useful for summarizing texts'
    )
]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

response = agent("What's the latest news about the Mars rover? Then please summarize the results.")
print(response['output'])


In [ ]:
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", n=2, best_of=2)

with get_openai_callback() as cb:
    result = llm("Tell me a joke")
    print(cb)

In [ ]:

# create our examples
examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    }, {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

# create an example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)


# load the model
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)

chain = LLMChain(llm=chat, prompt=few_shot_prompt_template)
chain.run("What's the capital of Democratic Republic of Congo?")

In [ ]:
template = """Question: {question}

Answer: """

prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)

# user question
question = "What is the capital city of France?"

In [ ]:
# initialize Hub LLM
hub_llm = HuggingFaceHub(
        repo_id='google/flan-t5-large',
    model_kwargs={'temperature':0}
)

# create prompt template > LLM chain
llm_chain = LLMChain(
    prompt=prompt,
    llm=hub_llm
)

# ask the user question about the capital of France
print(llm_chain.run(question))

In [ ]:
qa = [
    {'question': "What is the capital city of France?"},
    {'question': "What is the largest mammal on Earth?"},
    {'question': "Which gas is most abundant in Earth's atmosphere?"},
    {'question': "What color is a ripe banana?"}
]
res = llm_chain.generate(qa)
print( res )

In [ ]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=llm
)

qs_str = (
    "What is the capital city of France?\n" +
    "What is the largest mammal on Earth?\n" +
    "Which gas is most abundant in Earth's atmosphere?\n" +
	"What color is a ripe banana?\n"
)
llm_chain.run(qs_str)

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
summarization_template = "Summarize the following text to one sentence at maximum: {text}"
summarization_prompt = PromptTemplate(input_variables=["text"], template=summarization_template)
summarization_chain = LLMChain(llm=llm, prompt=summarization_prompt)
text = "LangChain provides many modules that can be used to build language model applications. Modules can be combined to create more complex applications, or be used individually for simple applications. The most basic building block of LangChain is calling an LLM on some input. Let’s walk through a simple example of how to do this. For this purpose, let’s pretend we are building a service that generates a company name based on what the company makes."
summarized_text = summarization_chain.predict(text=text)
# summarized_text = summarization_chain.run(text=text)

summarized_text

In [ ]:
translation_template = "Translate the following text from {source_language} to {target_language}: {text}"
translation_prompt = PromptTemplate(input_variables=["source_language", "target_language", "text"], template=translation_template)
translation_chain = LLMChain(llm=llm, prompt=translation_prompt)
source_language = "English"
target_language = "Spanish"
text = "I don't know what your name is or where do you live or anything whatsoever."
translated_text = translation_chain.predict(source_language=source_language, target_language=target_language, text=text)
translated_text

In [ ]:
# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

template = "You are an assistant that helps users find information about movies."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "Find information about the movie {movie_title}."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

response = chat(chat_prompt.format_prompt(movie_title="Inception").to_messages())

print(response.content)

In [ ]:
messages= chat_prompt.format_prompt(movie_title="Inception").to_messages()
messages.append(response)
prompt= HumanMessage(content= "Can you also recommend movies similar to this?")
messages.append(prompt)
response= chat(messages)
print(response.content)

In [ ]:
# Initialize language model
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)

# Load the summarization chain
summarize_chain = load_summarize_chain(llm)

# Load the document using PyPDFLoader
document_loader = PyPDFLoader(file_path="/home/jupyter/self_learning/Langchain/code/Resume_Naquib_Alam.pdf")
document = document_loader.load()

# Summarize the document
summary = summarize_chain(document)
print(summary['output_text'])

## Build a news Article Summariser

In [ ]:
import requests
from newspaper import Article

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_url = "https://www.artificialintelligence-news.com/2022/01/25/meta-claims-new-ai-supercomputer-will-set-records/"

session = requests.Session()

try:
    response = session.get(article_url, headers=headers, timeout=10)
    
    if response.status_code == 200:
        article = Article(article_url)
        article.download()
        article.parse()
        
        print(f"Title: {article.title}")
        print(f"Text: {article.text}")
        
    else:
        print(f"Failed to fetch article at {article_url}")
except Exception as e:
    print(f"Error occurred while fetching article at {article_url}: {e}")

In [ ]:
# we get the article data from the scraping part
article_title = article.title
article_text = article.text

# prepare template for prompt
template = """You are a very good assistant that summarizes online articles.

Here's the article you want to summarize.

==================
Title: {article_title}

{article_text}
==================

Write a summary of the previous article.
"""

prompt = template.format(article_title=article.title, article_text=article.text)

messages = [HumanMessage(content=prompt)]
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
summary= chat(messages)
print(summary.content)

In [ ]:
# prepare template for prompt
template = """You are a very good assistant that summarizes online articles.

Here's the article you want to summarize.

==================
Title: {article_title}

{article_text}
==================

Write a summary of the previous article in a bulleted list format.
"""

prompt = template.format(article_title=article.title, article_text=article.text)

messages = [HumanMessage(content=prompt)]
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
summary= chat(messages)
print(summary.content)

In [ ]:
# prepare template for prompt
template = """You are a very good assistant that summarizes online articles.

Here's the article you want to summarize.

==================
Title: {article_title}

{article_text}
==================

Write a summary of the previous article in a bulleted list format, in Spanish.
"""

prompt = template.format(article_title=article.title, article_text=article.text)

messages = [HumanMessage(content=prompt)]
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
summary= chat(messages)
print(summary.content)

In [ ]:
# we get the article data from the scraping part
article_title = article.title
article_text = article.text

# prepare template for prompt
template = """
As an advanced AI, you've been tasked to summarize online articles into bulleted points. Here are a few examples of how you've done this in the past:

Example 1:
Original Article: 'The Effects of Climate Change
Summary:
- Climate change is causing a rise in global temperatures.
- This leads to melting ice caps and rising sea levels.
- Resulting in more frequent and severe weather conditions.

Example 2:
Original Article: 'The Evolution of Artificial Intelligence
Summary:
- Artificial Intelligence (AI) has developed significantly over the past decade.
- AI is now used in multiple fields such as healthcare, finance, and transportation.
- The future of AI is promising but requires careful regulation.

Now, here's the article you need to summarize:

==================
Title: {article_title}

{article_text}
==================

{format_instructions}
"""


# create output parser class
class ArticleSummary(BaseModel):
    title: str = Field(description="Title of the article")
    summary: List[str] = Field(description="Bulleted list summary of the article")

    # validating whether the generated summary has at least three lines
    @validator('summary', allow_reuse=True)
    def has_three_or_more_lines(cls, list_of_lines):
        if len(list_of_lines) < 3:
            raise ValueError("Generated summary has less than three bullet points!")
        return list_of_lines

# set up output parser
parser = PydanticOutputParser(pydantic_object=ArticleSummary)

# template = """
# You are a very good assistant that summarizes online articles.

# Here's the article you want to summarize.

# ==================
# Title: {article_title}

# {article_text}
# ==================

# {format_instructions}
# """

prompt = PromptTemplate(
    template=template,
    input_variables=["article_title", "article_text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Format the prompt using the article title and text obtained from scraping
formatted_prompt = prompt.format_prompt(article_title=article_title, article_text=article_text)


# instantiate model class
model = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0.0)

# Use the model to generate a summary
output = model(formatted_prompt.to_string())

# Parse the output into the Pydantic model
parsed_output = parser.parse(output.split("\"]}")[0] + "\"]}")
print(parsed_output)

## Using the open-source GPT4ALL models locally

In [19]:
import requests
from pathlib import Path
from tqdm import tqdm

local_path = './models/llama-2-13b-chat.ggmlv3.q2_K.bin'
Path(local_path).parent.mkdir(parents=True, exist_ok=True)

# url = 'https://the-eye.eu/public/AI/models/nomic-ai/gpt4all/gpt4all-lora-quantized-ggml.bin'
url= "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/blob/main/llama-2-13b-chat.ggmlv3.q2_K.bin"

# send a GET request to the URL to download the file.
response = requests.get(url, stream=True)

# open the file in binary mode and write the contents of the response
# to it in chunks.
with open(local_path, 'wb') as f:
    for chunk in tqdm(response.iter_content(chunk_size=8192)):
        if chunk:
            f.write(chunk)

AttributeError: 'Response' object has no attribute 'download'

In [20]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])
# callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = GPT4All(model="./models/ggml-model-q4_0.bin", verbose=True)
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What happens when it rains somewhere?"
llm_chain.run(question)


gguf_init_from_file: invalid magic characters 'tjgg'
gguf_init_from_file: invalid magic characters 'tjgg'
load_gguf: gguf_init_from_file failed
magic_match: unsupported model architecture: 
gguf_init_from_file: invalid magic characters 'tjgg'


ValidationError: 1 validation error for GPT4All
__root__
  Unable to instantiate model: Model format not supported (no matching implementation found) (type=value_error)

In [ ]:
template = """Question: {question}

Answer: Let's answer in two sentence while being funny."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What happens when it rains somewhere?"
llm_chain.run(question)

## What other open-source models we can use

In [ ]:
# initialize Hub LLM
dolly3b_llm = HuggingFaceHub(repo_id='google/flan-t5-xl', model_kwargs={'temperature':0.1})

template = """Question: {question}

Answer: Let's answer in two sentence while being funny."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=dolly3b_llm)

question = "What happens when it rains somewhere?"
llm_chain.run(question)

https://learn.activeloop.ai/courses/take/langchain/multimedia/46317676-what-other-models-can-we-use-popular-llm-models-compared
__Various models to use with HuggingFace hub__: \
* repo_id='google/flan-t5-large/xl/xxl', model_kwargs={'temperature':0}
* repo_id='databricks/dolly-v2-12b/3b/7b', model_kwargs={'temperature':0}

## Using open-source models in Langchain

In [ ]:
import torch
from transformers import pipeline

generate_text = pipeline(model="meta-llama/Llama-2-7b-hf", torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map="auto", return_full_text=True)


In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

# template for an instrution with no input
prompt = PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}")

# template for an instruction with input
prompt_with_context = PromptTemplate(
    input_variables=["instruction", "context"],
    template="{instruction}\n\nInput:\n{context}")

hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

llm_chain = LLMChain(llm=hf_pipeline, prompt=prompt)
llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context)
print(llm_chain.predict(instruction="Explain to me the difference between nuclear fission and fusion.").lstrip())

In [ ]:
template = """Question: {question}

Answer: Let's answer in two sentence while being funny."""

prompt = PromptTemplate(template=template, input_variables=["question"])
question = "What happens when it rains somewhere?"
llm_chain.run(question)

# Keeping Knowledge organised with indexes

In [ ]:
# text to write to a local file
# taken from https://www.theverge.com/2023/3/14/23639313/google-ai-language-model-palm-api-challenge-openai
text = """Google opens up its AI language model PaLM to challenge OpenAI and GPT-3
Google is offering developers access to one of its most advanced AI language models: PaLM.
The search giant is launching an API for PaLM alongside a number of AI enterprise tools
it says will help businesses “generate text, images, code, videos, audio, and more from
simple natural language prompts.”

PaLM is a large language model, or LLM, similar to the GPT series created by OpenAI or
Meta’s LLaMA family of models. Google first announced PaLM in April 2022. Like other LLMs,
PaLM is a flexible system that can potentially carry out all sorts of text generation and
editing tasks. You could train PaLM to be a conversational chatbot like ChatGPT, for
example, or you could use it for tasks like summarizing text or even writing code.
(It’s similar to features Google also announced today for its Workspace apps like Google
Docs and Gmail.)
"""

# write text to local file
with open("my_file.txt", "w") as file:
    file.write(text)

# use TextLoader to load text from local file
loader = TextLoader("my_file.txt")
docs_from_file = loader.load()

print(len(docs_from_file))

# create a text splitter
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)

# split documents into chunks
docs = text_splitter.split_documents(docs_from_file)

print(len(docs))

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

chromadb_client = chromadb.Client()
db_openai = Chroma.from_documents(
    docs, embeddings, client=chromadb_client, collection_name="tmp_collection_1"
)

print("There are", db_openai._collection.count(), "in the collection")

retriever = db_openai.as_retriever()
model= OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
# create a retrieval chain
qa_chain = RetrievalQA.from_chain_type(llm= model, chain_type="stuff", retriever=retriever)
query = "How Google plans to challenge OpenAI?"
response = qa_chain.run(query)
print(response)



In [ ]:
# create compressor for the retriever
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever( base_compressor=compressor, base_retriever=retriever)

# retrieving compressed documents
retrieved_docs = compression_retriever.get_relevant_documents("How Google plans to challenge OpenAI?")
print(retrieved_docs[0].page_content)

## Exploring the world of embeddings

In [ ]:
## OpenAI Embedding
# Define the documents
print("=========OpenAI Embeddings===========")
documents = [
    "The cat is on the mat.",
    "There is a cat on the mat.",
    "The dog is in the yard.",
    "There is a dog in the yard.",
]

# Initialize the OpenAIEmbeddings instance
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Generate embeddings for the documents
document_embeddings = embeddings.embed_documents(documents)

# Perform a similarity search for a given query
query = "A cat is sitting on a mat."
query_embedding = embeddings.embed_query(query)

# Calculate similarity scores
similarity_scores = cosine_similarity([query_embedding], document_embeddings)[0]

# Find the most similar document
most_similar_index = np.argmax(similarity_scores)
most_similar_document = documents[most_similar_index]

print(f"Most similar document to the query '{query}':")
print(most_similar_document)


## HuggingFace Embedding
print("=========HuggingFace Embeddings===========")
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cuda:0'}
hf = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

document_embeddings = hf.embed_documents(documents)
query_embedding = hf.embed_query(query)

# Calculate similarity scores
similarity_scores = cosine_similarity([query_embedding], document_embeddings)[0]
print(similarity_scores)
# Find the most similar document
most_similar_index = np.argmax(similarity_scores)
most_similar_document = documents[most_similar_index]

print(f"Most similar document to the query '{query}':")
print(most_similar_document)

## Cohere Embedding
print("=========Cohere Embeddings===========")

# Initialize the CohereEmbeddings object
cohere = CohereEmbeddings(model="embed-multilingual-v2.0")

# Define a list of texts
docs= [
    "Hello from Cohere!", 
    "مرحبًا من كوهير!", 
    "Hallo von Cohere!",  
    "Bonjour de Cohere!", 
    "¡Hola desde Cohere!", 
    "Olá do Cohere!",  
    "Ciao da Cohere!", 
    "您好，来自 Cohere！", 
    "कोहेरे से नमस्ते!",
    "Paris is a very beautiful city.",
    "París es una ciudad muy hermosa.",
    "Paris est une très belle ville.",
    "पेरिस बहुत खूबसूरत शहर है."
]

# Generate embeddings for the texts
document_embeddings = cohere.embed_documents(docs)
query = ["Hello from Cohere!", "पेरिस बहुत खूबसूरत शहर है.",  "您好，来自 Cohere！"]
query_embedding = cohere.embed_query(query[1])
# Calculate similarity scores
similarity_scores = cosine_similarity([query_embedding], document_embeddings)[0]
sorted_similarity_scores_idx = np.argsort(-similarity_scores)[:5]
topk_docs= [docs[i] for i in sorted_similarity_scores_idx]
print(topk_docs)

## Build a customer support chatbot

In [ ]:
from langchain.document_loaders import SeleniumURLLoader
# we'll use information from the following articles
urls = ['https://beebom.com/what-is-nft-explained/',
        'https://beebom.com/how-delete-spotify-account/',
        'https://beebom.com/how-download-gif-twitter/',
        'https://beebom.com/how-use-chatgpt-linux-terminal/',
        'https://beebom.com/how-delete-spotify-account/',
        'https://beebom.com/how-save-instagram-story-with-music/',
        'https://beebom.com/how-install-pip-windows/',
        'https://beebom.com/how-check-disk-usage-linux/']

# use the selenium scraper to load the documents
loader = SeleniumURLLoader(urls=urls, browser= 'firefox')
docs_not_splitted = loader.load()

# we split the documents into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(docs_not_splitted)

# Before executing the following code, make sure to have
# your OpenAI key saved in the “OPENAI_API_KEY” environment variable.
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

chromadb_client = chromadb.Client()
db_openai = Chroma.from_documents(
    docs, embeddings, client=chromadb_client, collection_name="tmp_collection_2"
)

print("There are", db_openai._collection.count(), "in the collection")


# let's write a prompt for a customer support chatbot that
# answer questions using information extracted from our db
template = """You are an exceptional customer support chatbot that gently answer questions.

You know the following context information.

{chunks_formatted}

Answer to the following question from a customer. Use only information from the previous context information. Do not invent stuff.

Question: {query}

Answer:"""

prompt = PromptTemplate(
    input_variables=["chunks_formatted", "query"],
    template=template,
)

# the full pipeline

# user question
query = "How to check disk usage in linux?"

# retrieve relevant chunks
docs = db_openai.similarity_search(query)
retrieved_chunks = [doc.page_content for doc in docs]
print(len(retrieved_chunks))
# format the prompt
chunks_formatted = "\n\n".join(retrieved_chunks)
prompt_formatted = prompt.format(chunks_formatted=chunks_formatted, query=query)

# generate answer
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
answer = llm(prompt_formatted)
print(answer)

# Combining components together with Chains

## Chains and why they are used

### Sequential Chain

In [ ]:
from langchain.chains import SimpleSequentialChain
# poet
poet_template= """You are an American poet, your job is to come up with\
poems based on a given theme.

Here is the theme you have been asked to generate a poem on:
{input}\
"""

poet_prompt_template = PromptTemplate(input_variables=["input"], template=poet_template)

# creating the poet chain
poet_chain = LLMChain(llm=llm, output_key="poem", prompt=poet_prompt_template)

# critic
critic_template= """You are a critic of poems, you are tasked\
to inspect the themes of poems. Identify whether the poem includes romantic expressions or descriptions of nature.

Your response should be in the following format, as a Python Dictionary.
poem: this should be the poem you received 
Romantic_expressions: True or False
Nature_descriptions: True or False

Here is the poem submitted to you:
{poem}\
"""

critic_prompt_template = PromptTemplate(input_variables=["poem"], template=critic_template)

# creating the critic chain
critic_chain = LLMChain(llm=llm, output_key="critic_verified", prompt=critic_prompt_template)

overall_chain = SimpleSequentialChain(chains=[poet_chain, critic_chain])

# Run the poet and critic chain with a specific theme
theme= "the beauty of nature"
review = overall_chain.run(theme)

# Print the review to see the critic's evaluation
print(review)




### Custom Chain

In [ ]:
from langchain.chains import LLMChain
from langchain.chains.base import Chain
from typing import Dict, List


class ConcatenateChain(Chain):
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # Union of the input keys of the two chains.
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}

prompt_1 = PromptTemplate(
    input_variables=["word"],
    template="What is the meaning of the following word '{word}'?",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["word"],
    template="What is a word to replace the following: {word}?",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("artificial")
print(f"Concatenated output:\n{concat_output}")

## Create a Youtube video summarizer using Whisper and Langchain

In [ ]:
from pytube import YouTube
import whisper

# #Download youtube video
# # url = "https://www.youtube.com/watch?v=mBjPyte2ZZo"
# # url= "https://youtu.be/IuL-l2L_8Rk"
# url= "https://www.youtube.com/watch?v=QHfuiT1Y7Hw"
# savepath = '/home/jupyter/self_learning/Langchain/data/'
# YouTube(url).streams.filter(file_extension='mp4', res="144p").first().download(savepath)
# # YouTube(url).streams.first().download(savepath)


#load Whisper model and generate transcription of the video
savepath = '/home/jupyter/self_learning/Langchain/data/Yann LeCun on How to Fill the Gaps in Large Language Models.mp4'
model = whisper.load_model("base")
result = model.transcribe(savepath)
print(result['text'])

with open ('/home/jupyter/self_learning/Langchain/data/yann_lecun_transcription.txt', 'w') as file:  
    file.write(result['text'])


In [12]:
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
import textwrap

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
)

with open('/home/jupyter/self_learning/Langchain/data/yann_lecun_transcription.txt') as f:
    text = f.read()

texts = text_splitter.split_text(text)
docs = [Document(page_content=t) for t in texts[:10]]


chain = load_summarize_chain(llm, chain_type="map_reduce")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)


 Craig Smith interviews Jan LeCoon, a pioneer in deep learning and self-supervised learning,
discussing the limitations of large language models and Jan's new joint embedding predictive
architecture. Jan also shares his thoughts on consciousness and the potential for AI systems to
exhibit it. They discuss the use of self-supervised learning in natural language processing and its
potential for training neural networks to predict missing information in videos. The article also
explores the need for machines to learn about the world through observation and accumulate common
sense knowledge. The author proposes using self-supervised learning to teach machines how to predict
the future and plan actions accordingly. They also discuss the limitations of current language
models in understanding underlying reality and the potential for a joint embedding architecture to
address this. The article also delves into the speaker's work on developing new learning procedures
for complex neural networks

In [9]:
print(chain.llm_chain.prompt.template)

Write a concise summary of the following:


"{text}"


CONCISE SUMMARY:


In [16]:
prompt_template = """Write a concise bullet point summary of the following:


{text}


CONSCISE SUMMARY IN BULLET POINTS:"""

BULLET_POINT_PROMPT = PromptTemplate(template=prompt_template, 
                        input_variables=["text"])

chain = load_summarize_chain(llm, 
                             chain_type="stuff", 
                             prompt=BULLET_POINT_PROMPT)

output_summary = chain.run(docs)

wrapped_text = textwrap.fill(output_summary, 
                             width=1000,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

 

- Craig Smith interviews Jan LeCoon, a leading figure in deep learning and self-supervised learning.
- Jan discusses his new joint embedding predictive architecture and its potential to fill gaps in large language models.
- He also talks about his theory of consciousness and the potential for AI systems to exhibit conscious features.
- Jan explains the limitations of large language models and the need for machines to learn predictive world models.
- He suggests that incorporating self-supervised learning and planning capabilities into large language models could improve their performance.


In [17]:
chain = load_summarize_chain(llm, chain_type="refine")

output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)

 Craig Smith interviews Jan LeCoon, a pioneer in deep learning and self-supervised learning. They
discuss the limitations of large language models and Jan's new joint embedding predictive
architecture. Jan also shares his thoughts on consciousness and the potential for AI systems to
exhibit it. Jan is a professor at New York University and the Guarante Institute in the Centre for
Data Science, as well as the chief AI scientist at Fair, a fundamental AI research lab. They also
touch on the use of transformer architectures in self-supervised learning and its impact on natural
language processing. Jan explains that self-supervised learning involves training a large neural net
to predict missing words in a piece of text, resulting in improved text representations for
downstream tasks such as translation and content moderation. This technique has seen a revolution in
the past few years and is now widely used in practical applications. However, Jan's recent position
paper proposes using self

### Using RAG framework with chromadb for summarization

In [18]:
##Combine both transcript files in one text file
path= "/home/jupyter/self_learning/Langchain/data/"
filenames = ['yann_lecun_transcription.txt', 'thecottagefairy_transcription.txt']
with open(path+'yannlecun_thecottagefairy_comb_transcription.txt', 'w') as outfile:
    for fname in filenames:
        with open(path+fname) as infile:
            outfile.write(infile.read())

In [27]:
with open('/home/jupyter/self_learning/Langchain/data/yannlecun_thecottagefairy_comb_transcription.txt') as f:
    text = f.read()

# Split the documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0, separators=[" ", ",", "\n"]
    )
texts = text_splitter.split_text(text)
docs = [Document(page_content=t) for t in texts]
print(len(texts), len(docs))
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

60 60


In [29]:
chromadb_client = chromadb.Client()
db_openai = Chroma.from_documents(
    docs, embeddings, client=chromadb_client, collection_name="collection_transcript"
)

print("There are", db_openai._collection.count(), "in the collection")


There are 60 in the collection


In [56]:
retriever = db_openai.as_retriever()
# retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['k'] = 4

prompt_template = """Use the following pieces of transcripts from a video to answer the question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 chain_type_kwargs=chain_type_kwargs)

# print( qa.run("Summarize the mentions of google according to their AI program") )
# print( qa.run("what's lacking in large language models?") )
print( qa.run("Who is Yann Lecun?") )
# print( qa.run("What is the reason for Will Smith's popularity?") )
# print( qa.run("Who is Deepika Padukone?") ) ##How to restrict answers outside the transcript.
# print( qa.run("What are different ways of self-care?") )
# print( qa.run("What kind of videos she makes?") )
# print( qa.run("When did she start this youtube channel?") )

 Yann Lecun is a professor at New York University and at the Guarante Institute in the Centre for Data Science. He is also the chief AI scientist at Fair, which is the fundamental AI research lab at Facebook. He is a seminal figure in deep learning development and a long time proponent of self-supervised learning.


### How to restrict answers outside the transcript.

## Rough work

In [26]:
from langchain_mistralai.chat_models import ChatMistralAI

ModuleNotFoundError: No module named 'langchain_mistralai'